In [1]:
import pandas as pd
import os

from gensim.models import Nmf
from gensim.models.ldamulticore import LdaMulticore
from top2vec import Top2Vec
from bertopic import BERTopic

from gensim.corpora.dictionary import Dictionary

# loading processed dataframe, topic models, and BOW dictionary

In [2]:
def get_processed_df(csv_path):
    df = pd.read_csv(csv_path)
    return df

In [3]:
df = get_processed_df('process_csv_stage_2.csv')
df

,old_index,preprocessed_text,topic_lda,topic_nmf,topic_t2v,topic_btp
0,1,good morning name appreciate could help put st...,2,2,0,34
1,2,upgraded card tell agent upgrade anniversary d...,5,6,0,2
2,10,chase card report however fraudulent applicati...,2,4,0,16
3,11,try book ticket come across offer apply toward...,5,4,0,24
4,14,grand son give check deposit chase account fun...,6,1,0,0
...,...,...,...,...,...,...
21067,78303,chase card customer well decade offer multiple...,5,6,0,2
21068,78309,wednesday call chas visa credit card provider ...,8,5,0,39
21069,78310,familiar pay understand great risk provide con...,3,2,0,9
21070,78311,flawless credit chase credit card chase freedo...,4,0,0,6


In [4]:
# load 4 topic models
load_lda = LdaMulticore.load(os.path.join(os.getcwd(),'topic_models',"final_tune_lda"))
load_nmf = Nmf.load(os.path.join(os.getcwd(),'topic_models',"final_tune_nmf"))
load_t2v = Top2Vec.load(os.path.join(os.getcwd(),'topic_models','final_t2v'))
load_bertopic = BERTopic.load(os.path.join(os.getcwd(),'topic_models','final_bertopic'))
# load BOW dictionary for NMF and LDA topic models
load_lda_id2word = Dictionary.load(os.path.join(os.getcwd(),'topic_models',"final_tune_lda.id2word"))

# get topic distributions and topic names

## LDA

In [5]:
df.value_counts('topic_lda')

topic_lda
8    4316
0    4015
6    3443
5    1941
2    1887
1    1826
4    1467
7    1427
3     750
Name: count, dtype: int64

In [6]:
lda_topic_indexes = []
lda_topic_names = []
lda_topics = load_lda.show_topics(num_topics=-1, num_words=10,formatted=False)
for top in lda_topics:
    lda_topic_indexes.append(top[0])
    # process the topic names
    name = '_'.join([word[0] for word in top[1]])
    lda_topic_names.append(name)
lda_topic_df = pd.DataFrame({'topic_id':lda_topic_indexes,'topic_name':lda_topic_names})
lda_topic_df

,topic_id,topic_name
0,0,credit_card_account_payment_report_pay_late_cl...
1,1,charge_transaction_card_claim_fraud_account_ma...
2,2,information_request_letter_provide_state_repor...
3,3,bank_name_money_fraud_use_help_number_security...
4,4,payment_fee_pay_interest_balance_charge_make_r...
5,5,card_credit_dispute_receive_charge_purchase_of...
6,6,account_check_bank_fund_deposit_money_branch_c...
7,7,loan_mortgage_home_payment_property_modificati...
8,8,call_tell_would_say_get_ask_back_could_time_re...


## NMF

In [7]:
df.value_counts('topic_nmf')

topic_nmf
0    5237
2    3268
5    2896
4    2657
6    2313
1    1929
3    1802
7     970
Name: count, dtype: int64

In [8]:
nmf_topic_indexes = []
nmf_topic_names = []
nmf_topics = load_nmf.show_topics(num_topics=-1, num_words=10,formatted=False)
for top in nmf_topics:
    nmf_topic_indexes.append(top[0])
    # process the topic names
    name = '_'.join([word[0] for word in top[1]])
    nmf_topic_names.append(name)
nmf_topic_df = pd.DataFrame({'topic_id':nmf_topic_indexes,'topic_name':nmf_topic_names})
nmf_topic_df

,topic_id,topic_name
0,0,payment_pay_make_would_time_month_balance_rece...
1,1,check_would_tell_get_money_say_day_could_call_ask
2,2,bank_account_transaction_money_fund_fraud_clos...
3,3,account_close_open_check_deposit_fund_branch_t...
4,4,credit_report_consumer_debt_information_report...
5,5,call_receive_say_claim_send_number_letter_back...
6,6,card_charge_purchase_use_credit_fee_transactio...
7,7,loan_mortgage_service_home_modification_docume...


## Top2Vec

In [9]:
df.value_counts('topic_t2v')

topic_t2v
0    20234
1      240
2      240
3      125
4      112
5       77
6       44
Name: count, dtype: int64

In [10]:
t2v_topic_indexes = []
t2v_topic_names = []
t2v_topics_words, t2v_word_scores, t2v_topic_nums = load_t2v.get_topics()

t2v_topic_df = pd.DataFrame({'topic_id':list(t2v_topic_nums),'topic_name':['_'.join(top) for top in t2v_topics_words]})
t2v_topic_df

,topic_id,topic_name
0,0,cfpb_autopay_chase_wamu_overdrawn_garnishment_...
1,1,dispute_garnishment_creditor_defrauded_dispute...
2,2,creditor_autopay_creditors_debt_debts_credit_a...
3,3,autopay_cfpb_heloc_credit_refi_lender_refinanc...
4,4,autopay_refi_cfpb_lender_payment_payments_refi...
5,5,verified_unverified_chase_verification_fraudul...
6,6,usc_creditor_chase_garnishment_creditors_burea...


## BERTopic

In [11]:
df.value_counts('topic_btp')

topic_btp
 0     1923
-1     1044
 8      752
 1      708
 5      690
       ... 
 86      25
 85      23
 80      23
 88      19
 93      17
Name: count, Length: 95, dtype: int64

In [12]:
btp_topic_info = load_bertopic.get_topic_info()
btp_topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,10199,-1_the_to_and_my,"[the, to, and, my, that, was, chase, of, in, on]",NaN
1,0,1774,0_check_the_account_my,"[check, the, account, my, and, to, was, bank, ...",NaN
2,1,692,1_fees_overdraft_fee_account,"[fees, overdraft, fee, account, 3400, the, my ...",NaN
3,2,687,2_points_card_offer_the,"[points, card, offer, the, bonus, annual, for,...",NaN
4,3,544,3_inquiry_inquiries_credit_my credit,"[inquiry, inquiries, credit, my credit, hard, ...",NaN
...,...,...,...,...,...
90,89,15,89_car_the car_rental_the rental,"[car, the car, rental, the rental, the, charge...",NaN
91,90,15,90_identity_identity theft_theft_information,"[identity, identity theft, theft, information,...",NaN
92,91,15,91_loan_closing_the_and,"[loan, closing, the, and, to, you, documents, ...",NaN
93,92,15,92_tax_property tax_vehicle_lease,"[tax, property tax, vehicle, lease, property, ...",NaN


# Export topics to csv

In [13]:
lda_topic_df.to_csv('lda_topic.csv',index=False)
nmf_topic_df.to_csv('nmf_topic.csv',index=False)
t2v_topic_df.to_csv('t2v_topic.csv',index=False)
btp_topic_info.to_csv('btp_topic.csv',index=False)